In [ ]:
pip install mmcv-full==1.6.0
pip install mmengine==0.8.0
pip install mmsegmentation==0.29.0

In [1]:
try:
    import mmcv
    print("MMCV Version:", mmcv.__version__)
except ImportError:
    print("MMCV не установлен")
    
try:
    import mmengine
    print("MMEngine Version:", mmengine.__version__)
except ImportError:
    print("MMEngine не установлен")

try:
    import mmseg
    print("MMSegmentation Version:", mmseg.__version__)
except ImportError:
    print("MMSegmentation не установлен")


MMCV Version: 1.6.0
MMEngine Version: 0.8.0
MMSegmentation Version: 0.29.0


In [ ]:
import os
import requests
import zipfile
import torch
import mmcv
import numpy as np
import matplotlib.pyplot as plt
from mmseg.apis import inference_segmentor, init_segmentor
from mmcv import Config
import time

# Укажите рабочую директорию с обрабатываемыми изображениями
image_directory = r'C:\Users\anton\UII\Probka\объекты'

# URLs для скачивания
repo_url = 'https://github.com/open-mmlab/mmsegmentation/archive/refs/heads/master.zip'
repo_dir = 'mmsegmentation-master'
config_dir = os.path.join(repo_dir, 'configs')

# Функция для скачивания файла
def download_file(url, local_filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

# Скачиваем и распаковываем репозиторий, если он еще не скачан
if not os.path.isdir(repo_dir):
    zip_path = 'mmsegmentation.zip'
    download_file(repo_url, zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall()
    os.remove(zip_path)

# Получаем список конфигурационных файлов
config_files = []
for root, _, files in os.walk(config_dir):
    for file in files:
        if file.endswith('.py'):
            config_files.append(os.path.join(root, file))

# Список известных моделей и их URL
model_urls = {
    "upernet_beit-large_fp16_8x1_640x640_160k_ade20k": "https://download.openmmlab.com/mmsegmentation/v0.5/beit/upernet_beit-large_fp16_8x1_640x640_160k_ade20k/upernet_beit-large_fp16_8x1_640x640_160k_ade20k-8fc0dd5d.pth"
}

# Цвета для визуализации
color_mapping = {
    'ade20k': {0: (160, 32, 240), 3: (255, 0, 0), 5: (0, 128, 0)}
}

# Функция для предсказания и визуализации
def predict_and_visualize(model, image_path, dataset_name):
    # Предсказание
    image = mmcv.imread(image_path)
    start_time = time.time()
    result = inference_segmentor(model, image)
    end_time = time.time()
    prediction_time = end_time - start_time

    # Преобразование тензоров в numpy массивы
    seg_map = result[0].cpu().numpy() if isinstance(result[0], torch.Tensor) else result[0]

    # Визуализация
    plt.figure(figsize=(20, 10))

    # Оригинальное изображение
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.axis('off')
    plt.imshow(mmcv.bgr2rgb(image))  # Конвертация BGR в RGB

    # Визуализация результата сегментации
    color_seg = np.zeros((seg_map.shape[0], seg_map.shape[1], 3), dtype=np.uint8)
    for label, color in color_mapping[dataset_name].items():
        color_seg[seg_map == label, :] = color

    img = image * 0.7 + color_seg * 0.3
    img = img.astype(np.uint8)

    plt.subplot(1, 2, 2)
    plt.title('Segmented Image')
    plt.axis('off')
    plt.imshow(mmcv.bgr2rgb(img))  # Конвертация BGR в RGB

    plt.show()

    return prediction_time

# Функция для поиска всех изображений в заданной директории и её поддиректориях
def find_images_in_directory(directory):
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff']
    image_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                image_paths.append(os.path.join(root, file))
    return image_paths

# Поиск всех изображений в директории и её поддиректориях
image_paths = find_images_in_directory(image_directory)

summary = []

# Проход по всем конфигурациям и моделям
for config_file in config_files:
    config_path = config_file
    model_name = os.path.basename(config_file).split('.')[0]
    
    if model_name not in model_urls:
        continue

    checkpoint_url = model_urls[model_name]
    checkpoint_path = os.path.basename(checkpoint_url)

    # Скачиваем модель, если она еще не скачана
    if not os.path.isfile(checkpoint_path):
        try:
            download_file(checkpoint_url, checkpoint_path)
        except requests.HTTPError as e:
            print(f'Error downloading {checkpoint_url}: {e}')
            continue

    # Определяем имя набора данных
    if 'ade20k' in model_name:
        dataset_name = 'ade20k'
    else:
        continue

    # Загружаем модель один раз
    cfg = Config.fromfile(config_path)
    cfg.model.test_cfg.avg_non_ignore = True
    if 'evaluation' not in cfg:
        cfg.evaluation = dict(metric=['mIoU', 'mDice', 'mAcc', 'precision', 'recall'])
    model = init_segmentor(cfg, checkpoint_path, device='cuda:0' if torch.cuda.is_available() else 'cpu')

    # Применяем модель и визуализируем результат для каждого изображения
    for image_path in image_paths:
        try:
            prediction_time = predict_and_visualize(model, image_path, dataset_name)
            summary.append((model_name, image_path, prediction_time))
        except Exception as e:
            print(f'Error processing {image_path} with {config_file} and {checkpoint_path}: {e}')


In [ ]:
# Вывод времени обработкти изображений
for model_name, image_path, prediction_time in summary:
    print(f'Image: {image_path}, Prediction Time: {prediction_time:.2f}s')
